# Transpose Upper Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ x := U^T x $ where $ U $ is upper triangular.  Vector $ y $ is not to be touched (and, indeed, not even passed into the routines).  This is a little trickier than you might think.  You may want to do a few small problems by hand if you don't get the right answer.  Also, PictureFLAME may help you see what is going on.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

## The routine <br> <code> Trmv_ut_unb_var1( U, x ) </code>

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := U^T x $.  The "_ut_" in the name of the routine indicates this is the "upper, transpose" matrix-vector multiplication.

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.scal( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

<h1> <font color=red> NOTE: The algorithms for this operation march from bottom-right to top-left!!! </font> </h1>

In [1]:
import flame
import laff as laff

def Trmv_ut_unb_var1(U, x):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'BR')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'BOTTOM')

    while UBR.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'TL')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'TOP')

        #------------------------------------------------------------#

        laff.scal(upsilon11, chi1)
        laff.dots(u01, x0, chi1)

        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'BR')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'BOTTOM')

    flame.merge_2x1(xT, \
                    xB, x)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

U = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
xold = matrix( random.rand( 4,1 ) )

# Notice that U is not upper triangular.  We will only use the upper triangular part.

print( 'U before =' )
print( U )

print( 'x before =' )
print( x )

U before =
[[0.44765249 0.8573465  0.78638541 0.38948458]
 [0.26450979 0.55797843 0.53859708 0.97411089]
 [0.78210133 0.77825584 0.06664766 0.62107702]
 [0.59852516 0.29498456 0.41292664 0.59850429]]
x before =
[[0.51177597]
 [0.37783732]
 [0.11701564]
 [0.41527255]]


In [3]:
import numpy as np
laff.copy( x, xold )   # save the original vector x

Trmv_ut_unb_var1( U, x )

print( 'x after =' )
print( x )

#np.triu makes the matrix upper triangular
print( 'x - ( np.transpose( np.triu( U ) ) * xold ) = ' )
print( x - ( np.transpose( np.triu( U ) ) * xold ) )

x after =
[[0.22909779]
 [0.64959442]
 [0.61375406]
 [0.88860242]]
x - ( np.transpose( np.triu( U ) ) * xold ) = 
[[0.]
 [0.]
 [0.]
 [0.]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The routine <br> <code> Trmv_ut_unb_var2( U, x ) </code>

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := U^T x $.  The "_ut_" in the name of the routine indicates this is the "upper triangular, transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
<li> <code> laff.scal( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
def Trmv_ut_unb_var2(U, x):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'BR')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'BOTTOM')

    while UBR.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'TL')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'TOP')

        #------------------------------------------------------------#

        laff.axpy(chi1, u12t, x2)
        laff.scal(upsilon11, chi1)

        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'BR')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'BOTTOM')

    flame.merge_2x1(xT, \
                    xB, x)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

U = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
xold = matrix( random.rand( 4,1 ) )

# U is not upper triangular.  We will only use the upper triangular part.

print( 'U before =' )
print( U )

print( 'x before =' )
print( x )


U before =
[[0.69075904 0.94561476 0.8213862  0.3310621 ]
 [0.13570427 0.94730066 0.42706338 0.26469429]
 [0.58850547 0.49177378 0.62632947 0.2055342 ]
 [0.69648604 0.38827658 0.33949008 0.37566121]]
x before =
[[0.0188461 ]
 [0.52914872]
 [0.11762281]
 [0.3368143 ]]


In [6]:
laff.copy( x, xold )   # save the original vector y

Trmv_ut_unb_var2( U, x )

print( 'x after =' )
print( x )

#np.triu makes the matrix upper triangular
print( 'x - ( np.transpose( np.triu( U ) ) * xold ) = ' )
print( x - ( np.transpose( np.triu( U ) ) * xold ) )

x after =
[[0.01301812]
 [0.51908409]
 [0.31513061]
 [0.29700546]]
x - ( np.transpose( np.triu( U ) ) * xold ) = 
[[0.00000000e+00]
 [0.00000000e+00]
 [5.55111512e-17]
 [0.00000000e+00]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.